In [1]:
from gurobipy import GRB
import gurobipy as gb
import numpy as np

In [2]:
model = gb.Model("Sunnyshore_Bay_Optimization")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [25]:
months = ["May", "June", "July"]
month_end = ["May", "June", "July", "Aug"]
terms = [1, 2, 3]

In [26]:
# Decision Variables
B = {}
for i in months:
    for j in terms:
        B[i, j] = model.addVar(lb = 0, name="Borrow", vtype=GRB.CONTINUOUS)

cash_balances = {}
for m in month_end:
    cash_balances[m] = model.addVar(lb=0, name="Cash_balance", vtype=GRB.CONTINUOUS)

# Objective Function
interest_rates = {1: 0.0175, 2: 0.0225, 3: 0.0275}

# Set the objective function
model.setObjective(
    gb.quicksum(B[i, j] + B[i, j] * interest_rates[j] for i in months for j in terms),
    sense=GRB.MINIMIZE,
)

In [27]:
#Constraints on the borrow terms
constraint1 = model.addConstr(B["June", 3] == 0, "no_june_3_month")
constraint2 = model.addConstr(B["July", 2] == 0, "no_july_2_month")
constraint3 = model.addConstr(B["July", 3] == 0, "no_july_3_month")

# Add constraints
model.addConstr(cash_balances["May"] == 140000 + B["May", 1] + B["May", 2] + B["May", 3] + 180000 - 300000, "Cash Balance May")
model.addConstr(cash_balances["June"] == cash_balances["May"] + B["June", 1] + B["June", 2] - 1.0175*B["May", 1]+ 260000 - 400000, "Cash Balance June")
model.addConstr(cash_balances["July"] == cash_balances["June"] + B["July", 1] -1.0225*B["May", 2] - 1.0175*B["June", 1] + 420000- 350000, "Cash Balance July")
model.addConstr(cash_balances["Aug"] == cash_balances["July"] -1.0275*B["May", 3] -1.0225*B["June", 2] -1.0175*B["July", 1]+ 580000- 200000, "Cash Balance August")

# Cash balance Constraints
constraint4 = model.addConstr(cash_balances["May"] >= 25000, "Min Cash Balance May")
constraint5 = model.addConstr(cash_balances["June"] >= 20000, "Min Cash Balance June")
constraint6 = model.addConstr(cash_balances["July"] >= 35000, "Min Cash Balance July")
constraint7 = model.addConstr(cash_balances["Aug"] >= 18000, "Min Cash Balance August")

# Total amount borrowed constraints
constraint8 = model.addConstr(sum(B["May", t] for t in [1, 2, 3]) <= 250000, "Total Borrowing Limit May")
constraint9 = model.addConstr(sum(B["June", t] for t in [1, 2,3]) <= 150000, "Total Borrowing Limit June")
constraint10 = model.addConstr(sum(B["July", t] for t in [1,2,3]) <= 350000, "Total Borrowing Limit July")

# Cash balance ratio at the end of July constraint
constraint11 = model.addConstr(cash_balances["July"] >= 0.65 * (cash_balances["May"] + cash_balances["June"]), "Cash Balance Ratio July")

# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 63 rows, 52 columns and 151 nonzeros
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.080000e+05   0.000000e+00      0s
       7    1.4290473e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.429047297e+05


### (a) How many different investments can be made over the 4-month period?

In [21]:
print("Number of Investments:")
print(len(B))

Number of Investments:
9


### Exclude the investment in June for 3 months term and investments in July for 2 month and 3 month term. The total investments can be made are 6=9-3

### (b) Write down the cash balance constraint for money on-hand at the end of June

model.addConstr(cash_balances["June"] >= 20000, "Min Cash Balance June")

### (c) Write down the linear ratio constraint associated with the cash balance at the end of July

model.addConstr(cash_balances["July"] >= 0.65 * (cash_balances["May"] + cash_balances["June"]), "Cash Balance Ratio July")

### (d) What is the total amount that Sunnyshore Bay has to repay to the bank over the entire season

In [22]:
# Value of the objective function
print(f"Total Amount to Repay: {model.objVal}")

Total Amount to Repay: 380000.0


### (e) How much money does Sunnyshore Bay withdraw in May from all loans?

In [30]:
optimal_B11 = B["May",1].x
optimal_B12 = B["May",2].x
optimal_B13 = B["May",3].x

print(f"B11 = {optimal_B11}")
print(f"B12 = {optimal_B12}")
print(f"B13 = {optimal_B13}")

B11 = 0.0
B12 = 0.0
B13 = 5000.0


### Sunnyshore Bay withdraw a loan of $5000 with 3 month term in May

### (f) What is the cash balance at the end of August?

In [16]:
print("Aug Cash Balance:", cash_balances["Aug"])

Aug Cash Balance: <gurobi.Var C[m] (value 326963.3753071253)>


### (g) Due to potential unexpected repairs, one of the managers has suggested increasing the minimum cash balance for June to $27,500. How much will now have to be repaid if this change is approved?

In [10]:
constraint5 = model.addConstr(cash_balances["June"] >= 27500, "Min Cash Balance June")

# Solve the model
model.optimize()

print(f"Total Amount to Repay: {model.objVal}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18 rows, 13 columns and 41 nonzeros
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5053662e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.505366247e+05
Total Amount to Repay: 150536.6246928747


### (h) Formulate and solve the dual linear program demonstrating that the model you create is, indeed, the correct dual problem of the primal formulation.

In [209]:
# Check if the optimization was successful
if model.status == gb.GRB.OPTIMAL:
    # Get the optimal solution and objective value
    optimal_B11 = B["May",1].x
    optimal_B12 = B["May",2].x
    optimal_B13 = B["May",3].x
    optimal_B21 = B["June",1].x
    optimal_B22 = B["June",2].x
    optimal_B31 = B["July",1].x
    optimal_objective_value = model.objVal

    # Print the results
    print("Optimal Solution:")
    print(f"B11 = {optimal_B11}")
    print(f"B12 = {optimal_B12}")
    print(f"B13 = {optimal_B13}")
    print(f"B21 = {optimal_B21}")
    print(f"B22 = {optimal_B22}")
    print(f"B31 = {optimal_B31}")
    print("Optimal Objective Value:")
    print(f"z = {optimal_objective_value}")
    
    # These should equal the optimal solution to the dual problem
    print("Shadow Prices: ", (constraint4.pi, constraint5.pi, constraint6.pi, constraint7.pi, constraint8.pi, constraint9.pi, constraint10.pi, constraint11.pi))
else:
    print("No feasible solution found.")

Optimal Solution:
B11 = 0.0
B12 = 0.0
B13 = 5000.0
B21 = 54054.05405405405
B22 = 80945.94594594595
B31 = 0.0
Optimal Objective Value:
z = 142904.72972972973
Shadow Prices:  (0.0050000000000001155, 1.0175859950859951, 0.004914004914004844, 0.0, 0.0, 0.0, 0.0, 0.0)


In [191]:
model_p = gb.Model("Primal Problem")

In [192]:
# Primal variables
y1 = model_p.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y1")
y2 = model_p.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y2")
y3 = model_p.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y3")
y4 = model_p.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y4")
y5 = model_p.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y5")
y6 = model_p.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y6")
y7 = model_p.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y7")
y8 = model_p.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y8")

In [193]:

constraint4 = B["May", 1] + B["May", 2] + B["May", 3] >= 5000
constraint5 = -0.0175*B["May", 1]+ B["May", 2] + B["May", 3]  + B["June", 1] + B["June", 2] >= 140000
constraint6 = -0.0175*B["May", 1] -0.0225*B["May", 2]+ B["May", 3] - 0.0175*B["June", 1] + B["June", 2] + B["July", 1]  >= 85000
constraint7 = -0.0175*B["May", 1] - 0.0225*B["May", 2] - 0.0275*B["May", 3] - 0.0175*B["June", 1] - 0.0225*B["June", 2] - 0.0175*B["July", 1] >= -312000

constraint8 = -B["May", 1] - B["May", 2] - B["May", 3] >= -250000
constraint9 = -B["June", 1] -B["June", 2]>= -150000
constraint10 = -B["July", 1]>= -350000

constraint11 = -0.6561256*B["May",1] -1.3225*B["May",2] -0.3*B["May",3]-0.6675*B["June",1]+0.35*B["June",2]+B["July",1] >= -15000


In [194]:
# Set the objective function to maximize
model_p.setObjective(5000*y1 + 140000*y2 + 85000*y3 - 312000*y4-250000*y5 -150000*y6 - 350000*y7 -15000*y8, gb.GRB.MAXIMIZE)

In [195]:
coefficient_constraint = np.matrix([
    [1, 1, 1, 0, 0, 0],
    [-0.0175, 1, 1, 1, 1, 0],
    [-0.0175, -0.0225, 1, -0.0175, 1, 1],
    [-0.0175, -0.0225, -0.0275, -0.0175, -0.0225, -0.0175],
    [-1, -1, -1, 0, 0, 0],
    [0, 0, 0, -1, -1, 0],
    [0, 0, 0, 0, 0, -1],
    [-0.6561256, -1.3225, -0.3, -0.6675, 0.35, 1]
])

In [196]:
transposed_coefficient = coefficient_constraint.T
transposed_coefficient

matrix([[ 1.       , -0.0175   , -0.0175   , -0.0175   , -1.       ,
          0.       ,  0.       , -0.6561256],
        [ 1.       ,  1.       , -0.0225   , -0.0225   , -1.       ,
          0.       ,  0.       , -1.3225   ],
        [ 1.       ,  1.       ,  1.       , -0.0275   , -1.       ,
          0.       ,  0.       , -0.3      ],
        [ 0.       ,  1.       , -0.0175   , -0.0175   ,  0.       ,
         -1.       ,  0.       , -0.6675   ],
        [ 0.       ,  1.       ,  1.       , -0.0225   ,  0.       ,
         -1.       ,  0.       ,  0.35     ],
        [ 0.       ,  0.       ,  1.       , -0.0175   ,  0.       ,
          0.       , -1.       ,  1.       ]])

In [197]:
function_coefficient = np.array([1.0175,1.0225,1.0275,1.0175,1.0225,1.0175])
function_coefficient

array([1.0175, 1.0225, 1.0275, 1.0175, 1.0225, 1.0175])

In [198]:
dual_constraint1 = model_p.addConstr((y1-0.0175*y2 - 0.0175*y3 - 0.0175*y4 - y5-0.6561256*y8) <=1.0175)
dual_constraint2 = model_p.addConstr((y1+y2 - 0.0225*y3 - 0.0225*y4 - y5+1.3225*y8) <=1.0225)
dual_constraint3 = model_p.addConstr((y1+y2 +y3 - 0.0275*y4 - y5-0.3*y8) <=1.0275)
dual_constraint4 = model_p.addConstr((y2 -0.0175*y3 -0.0175*y4 - y6-0.6675*y8) <=1.0175)
dual_constraint5 = model_p.addConstr((y2 +y3 -0.0225*y4 - y6+0.35*y8) <=1.0225)
dual_constraint6 = model_p.addConstr((y3 -0.0175*y4 - y7+y8) <=1.0175)

In [199]:
model_p.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 8 columns and 32 nonzeros
Model fingerprint: 0x9014500d
Coefficient statistics:
  Matrix range     [2e-02, 1e+00]
  Objective range  [5e+03, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 6 rows, 7 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3000000e+35   9.925000e+30   2.300000e+05      0s
       6    1.4290473e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.429047297e+05


In [212]:
# Check if the optimization was successful
if model_p.status == gb.GRB.OPTIMAL:
    # Get the optimal solution and objective value for the dual problem
    optimal_y1 = y1.x
    optimal_y2 = y2.x
    optimal_y3 = y3.x
    optimal_y4 = y3.x
    optimal_y5 = y3.x
    optimal_y6 = y3.x
    optimal_y7 = y3.x
    optimal_y8 = y3.x
    optimal_dual_objective_value = model_p.objVal

    # Print the results
    print("Optimal Primal Solution:")
    print(f"y1 = {optimal_y1}")
    print(f"y2 = {optimal_y2}")
    print(f"y3 = {optimal_y3}")
    print(f"y4 = {optimal_y4}")
    print(f"y5 = {optimal_y5}")
    print(f"y6 = {optimal_y6}")
    print(f"y7 = {optimal_y7}")
    print(f"y8 = {optimal_y8}")
    print("Optimal Primal Objective Value:")
    print(f"Primal z = {optimal_dual_objective_value}")
    
    # These should equal the optimal solution to the primal problem
    print("Shadow Prices: ", (dual_constraint1.pi, dual_constraint2.pi, dual_constraint3.pi, dual_constraint4.pi, dual_constraint5.pi, dual_constraint6.pi))
else:
    print("No feasible solution found for the dual problem.")

Optimal Primal Solution:
y1 = 0.0050000000000001155
y2 = 1.0175859950859951
y3 = 0.004914004914004844
y4 = 0.004914004914004844
y5 = 0.004914004914004844
y6 = 0.004914004914004844
y7 = 0.004914004914004844
y8 = 0.004914004914004844
Optimal Primal Objective Value:
Primal z = 142904.72972972973
Shadow Prices:  (0.0, 0.0, 5000.0, 54054.05405405405, 80945.94594594595, 0.0)


In [211]:
# Print the results
print("Optimal Solution:")
print(f"B11 = {optimal_B11}")
print(f"B12 = {optimal_B12}")
print(f"B13 = {optimal_B13}")
print(f"B21 = {optimal_B21}")
print(f"B22 = {optimal_B22}")
print(f"B31 = {optimal_B31}")
print("Optimal Objective Value:")
print(f"z = {optimal_objective_value}")
    
# These should equal the optimal solution to the dual problem
print("Shadow Prices: ", (constraint4.pi, constraint5.pi, constraint6.pi, constraint7.pi, constraint8.pi, constraint9.pi, constraint10.pi, constraint11.pi))

Optimal Solution:
B11 = 0.0
B12 = 0.0
B13 = 5000.0
B21 = 54054.05405405405
B22 = 80945.94594594595
B31 = 0.0
Optimal Objective Value:
z = 142904.72972972973
Shadow Prices:  (0.0050000000000001155, 1.0175859950859951, 0.004914004914004844, 0.0, 0.0, 0.0, 0.0, 0.0)


### (i) Which formulation, the primal or the dual model, do you think is easier to solve?

#### The dual model is easier to solve as it has less decision variables 6 comparing to the primal model which has 8 decision variables